In [1]:
import scipy.stats as stats
import pandas as pd
import numpy as np

# random varaible values and parameter values
df = pd.read_csv('azdiabetes.csv')
X = df[['npreg', 'bp', 'bmi', 'ped', 'age']]
X = (X - X.mean()) / X.std()  # centering and scaling
X['intercept'] = 1
X = X.values
y = df['diabetes'].replace({'Yes': 1, 'No': 0})

/var/folders/dz/lgsl3l_d7fv4z40yrx07rfz80000gn/T/ipykernel_8713/3288299399.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y = df['diabetes'].replace({'Yes': 1, 'No': 0})


In [2]:
from sampler import GibbsSampler

g = len(y)
nu0 = 2
sigma02 = 1

GibbsSampler(y, X, g, nu0, sigma02).run_sampler()


(array([[-1.55530869e-02,  0.00000000e+00,  5.82113028e-03,
          7.78552911e-05,  1.25819572e-02,  6.11021673e-03],
        [ 1.63577394e-02,  0.00000000e+00, -2.84980698e-03,
          6.72179215e-03, -1.77842332e-02,  4.83343035e-03],
        [ 5.07802556e-03,  0.00000000e+00, -7.91988830e-03,
         -4.49727932e-04,  7.63337006e-03, -1.36354350e-03],
        ...,
        [-4.57718340e-03, -2.22908031e-03,  1.07198925e-02,
         -1.73771449e-02, -1.52029028e-02, -7.67060150e-03],
        [-2.27035877e-02, -2.22908031e-03, -1.23742546e-02,
         -1.12865551e-02,  1.71796292e-02, -7.51429554e-03],
        [-8.11624451e-04, -2.22908031e-03, -1.22373326e-02,
          6.61252279e-04,  6.30461927e-03,  6.39030174e-03]]),
 array([[1., 0., 1., 1., 1., 1.],
        [1., 0., 1., 1., 1., 1.],
        [1., 0., 1., 1., 1., 1.],
        ...,
        [1., 0., 1., 1., 1., 1.],
        [1., 0., 1., 1., 1., 1.],
        [1., 0., 1., 1., 1., 1.]]))

In [3]:
from sampler import MetropolisHastingsSampler

mean_beta = np.zeros(6)
cov_beta = np.diag([16, 4, 4, 4, 4, 4])
# Likelihood function
def likelihood(x):
    return np.exp(x) / (1 + np.exp(x))

MetropolisHastingsSampler(y, X, likelihood, mean_beta, cov_beta).run_sampler()


(array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        ...,
        [ 0.16096555,  0.00689069,  0.0445901 ,  0.11594117, -0.10461319,
         -0.15462605],
        [ 0.16096555,  0.00689069,  0.0445901 ,  0.11594117, -0.10461319,
         -0.15462605],
        [ 0.16096555,  0.00689069,  0.0445901 ,  0.11594117, -0.10461319,
         -0.15462605]]),
 array([[1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1.],
        [1., 0., 0., 0., 0., 0.],
        ...,
        [1., 0., 1., 1., 0., 1.],
        [1., 1., 1., 1., 0., 1.],
        [1., 1., 1., 1., 0., 1.]]))

In [4]:
from sampler import PolyaGamma

PolyaGamma(y, X, likelihood, mean_beta, cov_beta).run_sampler()

(array([[ 0.22712514, -0.06775521,  0.56148966,  0.32521387,  0.33018098,
         -0.66883394],
        [ 0.44194924,  0.04152797,  0.54265602,  0.25375708,  0.39920844,
         -0.75975442],
        [ 0.3606103 , -0.12001215,  0.70802174,  0.33131481,  0.53634537,
         -0.96630296],
        ...,
        [ 0.11662423, -0.14802876,  0.70269997,  0.536772  ,  0.49041515,
         -0.77037734],
        [ 0.12047153, -0.00917985,  0.65917242,  0.63967983,  0.47226946,
         -0.77709628],
        [ 0.22710861,  0.10525569,  0.53856861,  0.3361731 ,  0.51846628,
         -0.71626507]]),
 array([[1., 0., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 0., 1., 1., 1., 1.],
        ...,
        [1., 0., 1., 1., 1., 1.],
        [1., 0., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]]))